This Project we analyze a dataset from MarketCheck which holds information about the Used Car Market in the US.

Aggregated from over 65,000 dealer websites, the dataset contains ~7 million rows and 21 columns.

We'll be using a sample size of ~1 million rows with 15 columns to conduct our analysis

Through the analysis, we aim to:

1. Get to know the 'Used Car Market in the US'.

2. Understand the Used Car Market for newer used cars from years 2010-2021.

3. Find insights and value by answering questions to help find the best deal for a car.

In [1]:
#IMPORT LIBRARIES
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import pandas as pd
import statsmodels.api as sm

In [2]:
#IMPORT DATASET
df_used = pd.read_csv('us-dealers-used.csv')

/Users/hseo/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Number of rows and columns in a dataset
df_used.shape

(7104304, 21)

In [4]:
%%time
df_used = pd.read_csv('us-dealers-used.csv',  low_memory=False, nrows=2500000)

CPU times: user 12.3 s, sys: 6.87 s, total: 19.2 s
Wall time: 22.3 s


In [5]:
#Number of rows and columns used for analysis
df_used.shape

(2500000, 21)

In [6]:
df_used.head()

,id,vin,price,miles,stock_no,year,make,model,trim,body_type,...,drivetrain,transmission,fuel_type,engine_size,engine_block,seller_name,street,city,state,zip
0,38b2f52e-8f5d,1GCWGFCF3F1284719,20998.0,115879.0,W1T503168C,2015.0,Chevrolet,Express Cargo,Work Van,Cargo Van,...,RWD,Automatic,E85 / Unleaded,4.8,V,nissan ellicott city,8569 Baltimore National Pike,Ellicott City,MD,21043
1,97ba4955-ccf0,WBY7Z8C59JVB87514,27921.0,7339.0,P33243,2018.0,BMW,i3,s,Hatchback,...,RWD,Automatic,Electric / Premium Unleaded,0.6,I,hendrick honda pompano beach,5381 N Federal Highway,Pompano Beach,FL,33064
2,be1da9fd-0f34,ML32F4FJ2JHF10325,11055.0,39798.0,WM2091A,2018.0,Mitsubishi,Mirage G4,SE,Sedan,...,FWD,Automatic,Unleaded,1.2,I,russ darrow toyota,2700 West Washington St.,West Bend,WI,53095
3,84327e45-6cb6,1GCPTEE15K1291189,52997.0,28568.0,9U2Y425A,2019.0,Chevrolet,Colorado,ZR2,Pickup,...,4WD,Automatic,Diesel,2.8,I,young kia,308 North Main Street,Layton,UT,84041
4,cde691c3-91dd,1G2AL18F087312093,NaN,188485.0,T36625A,2008.0,Pontiac,G5,Base,Coupe,...,FWD,Automatic,Unleaded,2.2,I,pappas toyota,10011 Spencer Rd,Saint Peters,MO,63376


In [7]:
df_used.dtypes

id               object
vin              object
price           float64
miles           float64
stock_no         object
year            float64
make             object
model            object
trim             object
body_type        object
vehicle_type     object
drivetrain       object
transmission     object
fuel_type        object
engine_size     float64
engine_block     object
seller_name      object
street           object
city             object
state            object
zip              object
dtype: object

We will selecting 15 columns out of the 21 columns that are relevant for our analysis

In [8]:
df_used = df_used[['vin', 'price', 'miles','year','make', 'model', 'body_type', 'vehicle_type',
                   'drivetrain', 'transmission', 'fuel_type', 'engine_size', 'city', 'state', 'zip']]

In [9]:
df_used.shape

(2500000, 15)

Next we check for missing values and duplicates 

In [10]:
df_used.isnull().sum()

vin                  0
price           229615
miles            25051
year                72
make                 0
model             4015
body_type        13922
vehicle_type     19210
drivetrain        7870
transmission      6744
fuel_type        22893
engine_size      51688
city              4208
state             4215
zip               4295
dtype: int64

In [11]:
df_used = df_used.dropna()

In [12]:
df_used.isnull().sum()

vin             0
price           0
miles           0
year            0
make            0
model           0
body_type       0
vehicle_type    0
drivetrain      0
transmission    0
fuel_type       0
engine_size     0
city            0
state           0
zip             0
dtype: int64

The Vehicle Identification Number (VIN) is a unique identifier for each individual car. We will check to see if duplicates of the same car are listed 

In [13]:
df_used.drop_duplicates(subset=['vin'],keep='first')

,vin,price,miles,year,make,model,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,city,state,zip
0,1GCWGFCF3F1284719,20998.0,115879.0,2015.0,Chevrolet,Express Cargo,Cargo Van,Truck,RWD,Automatic,E85 / Unleaded,4.8,Ellicott City,MD,21043
1,WBY7Z8C59JVB87514,27921.0,7339.0,2018.0,BMW,i3,Hatchback,Car,RWD,Automatic,Electric / Premium Unleaded,0.6,Pompano Beach,FL,33064
2,ML32F4FJ2JHF10325,11055.0,39798.0,2018.0,Mitsubishi,Mirage G4,Sedan,Car,FWD,Automatic,Unleaded,1.2,West Bend,WI,53095
3,1GCPTEE15K1291189,52997.0,28568.0,2019.0,Chevrolet,Colorado,Pickup,Truck,4WD,Automatic,Diesel,2.8,Layton,UT,84041
6,1B7HC16Y8YS543285,3995.0,137537.0,2000.0,Dodge,Ram Pickup,Pickup,Truck,RWD,Manual,Unleaded,5.2,Fremont,OH,43420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499986,19XFB2F84CE372899,12990.0,91941.0,2012.0,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Pomona,CA,91768
2499990,19XFB2F5XFE701616,14988.0,59075.0,2015.0,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Westport,MA,02790
2499994,2HGFA1F62BH541849,6995.0,195369.0,2011.0,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Medford,OR,97501
2499995,2HGFA16828H532613,6995.0,107394.0,2008.0,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Lowell,MA,01852


In [14]:
df_used.shape

(2213774, 15)

In [15]:
df_used.dtypes

vin              object
price           float64
miles           float64
year            float64
make             object
model            object
body_type        object
vehicle_type     object
drivetrain       object
transmission     object
fuel_type        object
engine_size     float64
city             object
state            object
zip              object
dtype: object

In [16]:
df_used['year'] = df_used['year'].astype(int)

In [17]:
df_used.dtypes

vin              object
price           float64
miles           float64
year              int64
make             object
model            object
body_type        object
vehicle_type     object
drivetrain       object
transmission     object
fuel_type        object
engine_size     float64
city             object
state            object
zip              object
dtype: object

In [18]:
df_used['vin'].duplicated().sum()

1141960

In [19]:
df_used.vin.duplicated().sum()

1141960

In [20]:
(~df_used.vin.duplicated()).sum()

1071814

In [21]:
df_used.loc[df_used.duplicated(), :]

,vin,price,miles,year,make,model,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,city,state,zip
52,1B7HC16Y8YS543285,3995.0,137537.0,2000,Dodge,Ram Pickup,Pickup,Truck,RWD,Manual,Unleaded,5.2,Fremont,OH,43420
1307,WDDUG8FB2HA324924,68822.0,30109.0,2017,Mercedes-Benz,S-Class,Sedan,Car,4WD,Automatic,Premium Unleaded,4.7,Sycamore,IL,60178
1597,ZAM45VMA3H0194436,89895.0,17791.0,2017,Maserati,GranTurismo,Convertible,Car,RWD,Automatic,Premium Unleaded,4.7,Great Neck,NY,11021
1638,3C6TR5EJ6JG222709,51995.0,69156.0,2018,RAM,Ram 2500 Pickup,Pickup,Truck,4WD,Automatic,Unleaded,6.4,Corpus Christi,TX,78415
1641,1GNDS13S452197465,4911.0,160000.0,2005,Chevrolet,TrailBlazer,SUV,Truck,RWD,Automatic,Unleaded,4.2,Jackson,TN,38305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499978,2HGFB2F57CH557993,10350.0,62529.0,2012,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Wilkes-Barre,PA,18702
2499979,2HGFA1F91BH521473,11800.0,62000.0,2011,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Auburn,NY,13021
2499982,2HGFB2E54FH534547,11690.0,84772.0,2015,Honda,Civic,Sedan,Car,FWD,Manual,Unleaded,1.8,Buford,GA,30519
2499985,2HGFB2F53FH552309,14791.0,46623.0,2015,Honda,Civic,Sedan,Car,FWD,Automatic,Unleaded,1.8,Wilmington,NC,28405


In [22]:
df_used.vin.drop_duplicates(inplace=True)

In [23]:
df_used.vin.duplicated().sum()

1141960

In [24]:
df_used.drop_duplicates(subset=['vin'], inplace=True)

In [25]:
df_used.vin.duplicated().sum()

0

In [26]:
df_used.shape

(1071814, 15)

In [27]:
df_used = df_used.loc[(df_used['year'] > 2009) & (df_used['year'] < 2022) ]

In [28]:
df_used.shape

(995980, 15)

In [38]:
range_yrs = df_used.year.unique()
range_yrs.sort()
print(range_yrs)

[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
